<a href="https://colab.research.google.com/github/rotom303/Final_Project/blob/Aakash/FPP_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Set up the notebook environment
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import scipy
import random
from scipy.stats import pearsonr
from scipy import signal as sig

In [2]:
#Your code here
from google.colab import drive
drive.mount('/content/drive') 

%cd drive/MyDrive/Penn/'Spring 2023'/'BE 5210'/Project/Part_2

Mounted at /content/drive
/content/drive/MyDrive/Penn/Spring 2023/BE 5210/Project/Part_2


In [3]:
NUM_FINGERS = 5

In [4]:
proj_data = scipy.io.loadmat('raw_training_data.mat')

In [5]:
#splitting into testing and training sets
train_data_proportion = 0.7
#patient 1

random.seed(123)

#glove
total_p1_glove=proj_data['train_dg'][0][0]
training_rows = list(range(0, int(len(total_p1_glove)*train_data_proportion)))
training_p1_glove=total_p1_glove[training_rows,:] #60% of samples in training
testing_p1_glove=np.delete(total_p1_glove, training_rows, axis=0) #40% of samples in testing

#ecog
total_p1_ecog=proj_data['train_ecog'][0][0]
# total_p1_ecog=np.delete(total_p1_ecog, 54, 1)
training_rows = list(range(0, int(len(total_p1_ecog)*train_data_proportion)))
training_p1_ecog=total_p1_ecog[training_rows,:] #60% of samples in training
testing_p1_ecog=np.delete(total_p1_ecog, training_rows, axis=0) #60% of samples in training

#patient 2

#glove
total_p2_glove=proj_data['train_dg'][1][0]
training_rows = list(range(0, int(len(total_p2_glove)*train_data_proportion)))
training_p2_glove=total_p2_glove[training_rows,:] #60% of samples in training
testing_p2_glove=np.delete(total_p2_glove, training_rows, axis=0) #60% of samples in training
#ecog
total_p2_ecog=proj_data['train_ecog'][1][0]
# total_p2_ecog=np.delete(total_p2_ecog, [20, 37], 1)
training_rows = list(range(0, int(len(total_p2_ecog)*train_data_proportion)))
training_p2_ecog=total_p2_ecog[training_rows,:] #60% of samples in training
testing_p2_ecog=np.delete(total_p2_ecog, training_rows, axis=0) #60% of samples in training

#patient 3

#glove
total_p3_glove=proj_data['train_dg'][2][0]
training_rows = list(range(0, int(len(total_p3_glove)*train_data_proportion)))
training_p3_glove=total_p3_glove[training_rows,:] #60% of samples in training
testing_p3_glove=np.delete(total_p3_glove, training_rows, axis=0) #60% of samples in training
#ecog
total_p3_ecog=proj_data['train_ecog'][2][0]
training_rows = list(range(0, int(len(total_p3_ecog)*train_data_proportion)))
training_p3_ecog=total_p3_ecog[training_rows,:] #60% of samples in training
testing_p3_ecog=np.delete(total_p3_ecog, training_rows, axis=0) #60% of samples in training

In [6]:
def filter_data(raw_eeg, fs=1000, cutoffs=[75, 200]):
  """
  Write a filter function to clean underlying data.
  Filter type and parameters are up to you. Points will be awarded for reasonable filter type, parameters and application.
  Please note there are many acceptable answers, but make sure you aren't throwing out crucial data or adversly
  distorting the underlying data!

  Input: 
    raw_eeg (samples x channels): the raw signal
    fs: the sampling rate (1000 for this dataset)
  Output: 
    clean_data (samples x channels): the filtered signal
  """
  number_of_channels = np.shape(raw_eeg)[1] #number of channels
  filteredData = np.zeros(np.shape(raw_eeg)); #filtered data output

  #butterworth filter of 4th order
  sos = sig.butter(4, cutoffs, 'bandpass', analog=False, fs=fs, output='sos'); # returns filter coefficients

  #for each channel
  for chanInd in np.arange(number_of_channels):
    # subtract mean from each datapoint
    currFilt = raw_eeg[:, chanInd] - np.mean(raw_eeg[:, chanInd]);
    currFilt = sig.sosfiltfilt(sos, currFilt) # forward-backward digital filter using cascaded second-order sections                                
    filteredData[:, chanInd] = currFilt
  return filteredData

In [7]:
def NumWins(x, winLen, winDisp, fs):

    # Calculate the length of the signal in samples
    xLen = len(x)
    # Calculate the number of windows
    windows = ((len(x)/fs)/ winDisp) - (winLen / winDisp) + (winDisp / winDisp) - ((((len(x)/fs) - winLen + winDisp) % winDisp)/winDisp)
    #windows = ((len(x)/fs)/ winDisp) - (winLen / winDisp) + (winDisp / winDisp)
    return int(windows)

In [8]:
def line_length(signal):
    return np.sum(np.absolute(np.ediff1d(signal))) 

def zero_crossings(x):
  xbar=np.mean(x)
  number_of_crossings = 0
  for i in range(1,len(x)):
    if (x[i-1]-xbar) > 0 and (x[i]-xbar) < 0:
      number_of_crossings += 1
    elif (x[i-1]-xbar) < 0 and (x[i]-xbar) > 0:
      number_of_crossings += 1
  return number_of_crossings

def energy(signal):
  return np.sum(np.square(signal))

def area(signal):
  return np.sum(np.absolute(signal))

def time_avg(signal):
  return np.mean(signal)

from numpy.fft import fft, ifft

def band_power(signal,fs,low,high):
  X = fft(signal)
  N = len(X)
  n = np.arange(N)
  T = N/fs #sampling rate=1000
  freq = n/T 
  power_spectrum=np.abs(X)

  # Find values in frequency vector corresponding to input band
  index_band = np.logical_and(freq >= low, freq <= high)
  #average frequency domain magnitude
  avg_mag=np.mean(power_spectrum[index_band])

  return avg_mag

In [9]:
def get_features(filtered_window, fs=1000):
  """
    Write a function that calculates features for a given filtered window. 
    Feel free to use features you have seen before in this class, features that
    have been used in the literature, or design your own!

    Input: 
      filtered_window (window_samples x channels): the window of the filtered ecog signal 
      fs: sampling rate
    Output:
      features (channels x num_features): the features calculated on each channel for the window
  """
  #print(np.shape(filtered_window))
  [window_samples,num_channels]=np.shape(filtered_window)

  features=np.empty((num_channels,5))

  for cc in range(num_channels):

    curr_window=filtered_window[:,cc] 

    #features[cc,0]=line_length(curr_window)
    #features[cc,0]=area(curr_window)
    features[cc,0]=energy(curr_window) # Energy in signal window
    #features[cc,3]=zero_crossings(curr_window)
    features[cc,1]=band_power(curr_window,1000,8,12) # average frequency-domain magnitude in alpha frequency band
    features[cc,2]=band_power(curr_window,1000,75,95) # average frequency-domain magnitude in low gamma frequency band
    features[cc,3]=band_power(curr_window,1000,96,115) # average frequency-domain magnitude in high gamma frequency band
    features[cc,4]=time_avg(curr_window) #Average time-domain voltage

  return features

In [10]:
def get_windowed_feats(raw_ecog, fs, window_length, window_overlap):
  """
    Write a function which processes data through the steps of filtering and
    feature calculation and returns features. Points will be awarded for completing
    each step appropriately (note that if one of the functions you call within this script
    returns a bad output, you won't be double penalized). Note that you will need
    to run the filter_data and get_features functions within this function. 

    Inputs:
      raw_eeg (samples x channels): the raw signal
      fs: the sampling rate (1000 for this dataset)
      window_length: the window's length
      window_overlap: the window's overlap
    Output: 
      all_feats (num_windows x (channels x features)): the features for each channel for each time window
        note that this is a 2D array. 
  """
  clean_data=filter_data(raw_ecog, fs)
  [num_samples,num_channels]=np.shape(clean_data)

  num_windows = NumWins(clean_data[:,0], window_length,window_overlap, fs) #calculate number of windows and remaining time

  #convert everything to units of samples
  num_samples = len(clean_data)
  winLen_samples=round(window_length*fs) #window length in samples
  winDisp_samples=round(window_overlap*fs) #window displacement in samples
  
  # empty list to store features
  feature_vector = []
  # The end of the first window is the last sample
  window_end = num_samples
  # List to store tuples containing start and end indices for each slice

  for i in range(round(num_windows)):

    # Compute the start of the window using the end of the window
    window_start = window_end - winLen_samples
    # If the number of samples left is not sufficient, break out of the loop
    if window_start < 0:
        break
    
    signal_window=clean_data[window_start:window_end,:] #signal in that window
    
    # store feature of that window
    feature_vector.append(get_features(signal_window, fs).flatten('F'))

    #set new ending index of window
    window_end -= winDisp_samples 
  
  # Reverse at the end since we are looking at the windows backwards
  feature_vector.reverse()


  return feature_vector

In [11]:
def create_R_matrix(features, N_wind):
  """
  Write a function to calculate the R matrix

  Input:
    features (samples (number of windows in the signal) x channels x features): 
      the features you calculated using get_windowed_feats
    N_wind: number of windows to use in the R matrix

  Output:
    R (samples x (N_wind*channels*features))
  """
  padded_features = np.copy(features)
  for i in list(range(N_wind-2, -1, -1)):
      a = features[i]
      padded_features = np.vstack([a, padded_features])
  samples = len(features)   # number of rows = number of windows

  R = np.zeros((samples, 1+(N_wind*len(features[0,:]))))  # len(features[0,:]) = (num of features)*(num of channels)
  lst = np.array(list(range(1, 1+N_wind)))
  R[:, 0] = 1

  for i in range(len(features[0,:])):   # goes thru each column of the features matrix
    for j in range(len(lst)):
        x = lst[j]
        R[:, x] = padded_features[j : (len(padded_features)-(N_wind-1-j)), i]
    lst = lst + N_wind
  return R

In [12]:
def standardize_train_test(feature_matrix,test_feature_matrix):

  [windows,feats]=np.shape(feature_matrix)
  [windows_test,feats_test]=np.shape(test_feature_matrix)
  normFeats=np.empty((windows,feats))
  testnormFeats=np.empty((windows_test,feats_test))

  for i in range(feats):
      curr_train = feature_matrix[:,i]
      mean_train=np.mean(curr_train)
      std_train=np.std(curr_train)
      normFeats[:,i]=(curr_train-mean_train)/std_train
      curr_test = test_feature_matrix[:,i]
      mean_test=np.mean(curr_test)
      std_test=np.std(curr_test)
      testnormFeats[:,i]=(curr_test-mean_train)/std_train #normalize test features using training mean/std

  return normFeats,testnormFeats

In [13]:
from numpy.linalg import inv

## Patient 1

In [14]:
window_length=100e-3
window_overlap=50e-3
fs=1000

#calculate features for training data, call it p1_trainFeats
p1_trainFeats=get_windowed_feats(training_p1_ecog, fs, window_length, window_overlap)

p1_trainFeats = np.array(p1_trainFeats)
#calculate R for training data
R_train=create_R_matrix(p1_trainFeats,3) #N=3


#calculate Y for the training data

M=NumWins(training_p1_glove[:,0],window_length,window_overlap, fs) #calculate number of windows and remaining time

#convert everything to units of samples 
winLen_samples=round(window_length*fs) #window length in samples
winDisp_samples=round(window_overlap*fs) #window displacement in samples 

# ensuring right alignment
window_start_in_windows = round(len(training_p1_glove[:,0])/fs) - (M*window_overlap)
window_start = round(window_start_in_windows*fs)

Y_train_p1=np.empty([int(M),5])

for i in range(round(M)): #for each window
  Y_train_p1[i,:]=training_p1_glove[window_start,:] #find the flexion data for each finger at that window from glove data for P1
  window_start=window_start+winDisp_samples #move to next window

#calculate Y for the test data- basically flexion data downsampled to be Y=Mx5

M=NumWins(testing_p1_glove[:,0],window_length,window_overlap, fs) #calculate number of windows and remaining time

#convert everything to units of samples
winLen_samples=round(window_length*fs) #window length in samples
winDisp_samples=round(window_overlap*fs) #window displacement in samples 

#starting index of x- first window occurs after remainder samples since right aligned
window_start_in_windows = round(len(testing_p1_glove[:,0])/fs) - (M*window_overlap)
window_start = round(window_start_in_windows*fs)

#initialize
Y_test_p1=np.empty([int(M),5])

for mm in range(round(M)): #for each window
  Y_test_p1[mm,:]=testing_p1_glove[window_start,:] #find the flexion data for each finger at that window from glove data for P1
  window_start=window_start+winDisp_samples #move to next window

# Filter for subject 1
f1=np.matmul(inv(np.matmul(np.transpose(R_train),R_train)),np.matmul(np.transpose(R_train),Y_train_p1))

## Patient 2

In [15]:

#calculate features for training data, call it p2_trainFeats
p2_trainFeats = get_windowed_feats(training_p2_ecog, fs, window_length, window_overlap)

p2_trainFeats = np.array(p2_trainFeats)
R_train=create_R_matrix(p2_trainFeats,3) #N=3

#calculate Y for the training data
M=NumWins(training_p2_glove[:,0],window_length,window_overlap, fs) #calculate number of windows and remaining time

#convert everything to units of samples 
winLen_samples=round(window_length*fs) #window length in samples
winDisp_samples=round(window_overlap*fs) #window displacement in samples 

# ensuring right alignment
window_start_in_windows = round(len(training_p2_glove[:,0])/fs) - (M*window_overlap)
window_start = round(window_start_in_windows*fs)

#initialize
Y_train_p2=np.empty([int(M),5])

for i in range(round(M)): #for each window
  Y_train_p2[i,:]=training_p2_glove[window_start,:] #find the flexion data for each finger at that window from glove data for p2
  window_start=window_start+winDisp_samples #move to next window

#calculate Y for the test data- basically flexion data downsampled to be Y=Mx5

M=NumWins(testing_p2_glove[:,0],window_length,window_overlap, fs) #calculate number of windows and remaining time

winLen_samples=round(window_length*fs) #window length in samples
winDisp_samples=round(window_overlap*fs) #window displacement in samples 
window_start_in_windows = round(len(testing_p2_glove[:,0])/fs) - (M*window_overlap)
window_start = round(window_start_in_windows*fs)

Y_test_p2=np.empty([int(M),5])

for i in range(round(M)): #for each window
  Y_test_p2[i,:]=testing_p2_glove[window_start,:] #find the flexion data for each finger at that window from glove data for p2
  window_start=window_start+winDisp_samples #move to next window

# Filter for subject 2
f2=np.matmul(inv(np.matmul(np.transpose(R_train),R_train)),np.matmul(np.transpose(R_train),Y_train_p2))

## Patient 3

In [16]:

#calculate features for training data, call it p3_trainFeats
p3_trainFeats=get_windowed_feats(training_p3_ecog, fs, window_length, window_overlap)
p3_trainFeats = np.array(p3_trainFeats)
R_train=create_R_matrix(p3_trainFeats,3) #N=3


#calculate Y for the training data
M=NumWins(training_p3_glove[:,0],window_length,window_overlap, fs) #calculate number of windows

winLen_samples=round(window_length*fs) #window length in samples
winDisp_samples=round(window_overlap*fs) #window displacement in samples 
window_start_in_windows = round(len(training_p3_glove[:,0])/fs) - (M*window_overlap)
window_start = round(window_start_in_windows*fs)

Y_train_p3=np.empty([int(M),5])

for i in range(round(M)): #for each window
  Y_train_p3[i,:]=training_p3_glove[window_start,:] #find the flexion data for each finger at that window from glove data for p3
  window_start=window_start+winDisp_samples #move to next window

M=NumWins(testing_p3_glove[:,0],window_length,window_overlap, fs) #calculate number of windows

#convert everything to units of samples
winLen_samples=round(window_length*fs) #window length in samples
winDisp_samples=round(window_overlap*fs) #window displacement in samples 

#starting index of x- first window occurs after remainder samples since right aligned
window_start_in_windows = round(len(testing_p3_glove[:,0])/fs) - (M*window_overlap)
window_start = round(window_start_in_windows*fs)

#initialize
Y_test_p3=np.empty([int(M),5])

for i in range(round(M)): #for each window
  Y_test_p3[i,:]=testing_p3_glove[window_start,:] #find the flexion data for each finger at that window from glove data for p3
  window_start=window_start+winDisp_samples #move to next window

# Filter for subject 3
f3=np.matmul(inv(np.matmul(np.transpose(R_train),R_train)),np.matmul(np.transpose(R_train),Y_train_p3))

In [17]:
# Get the features on the testing data
test_features_p1 = get_windowed_feats(testing_p1_ecog, fs, window_length, window_overlap)
test_features_p2 = get_windowed_feats(testing_p2_ecog, fs, window_length, window_overlap)
test_features_p3 = get_windowed_feats(testing_p3_ecog, fs, window_length, window_overlap)
test_features_p1 = np.array(test_features_p1)
test_features_p2 = np.array(test_features_p2)
test_features_p3 = np.array(test_features_p3)

### Standardize All Data + Create R Matrices

In [18]:
normalized_train_p1,normalized_test_p1 = standardize_train_test(p1_trainFeats,test_features_p1)
normalized_train_p2,normalized_test_p2 = standardize_train_test(p2_trainFeats,test_features_p2)
normalized_train_p3,normalized_test_p3 = standardize_train_test(p3_trainFeats,test_features_p3)
R_train1 = create_R_matrix(normalized_train_p1,3)
R_train2 = create_R_matrix(normalized_train_p2,3)
R_train3 = create_R_matrix(normalized_train_p3,3)
R_test1 = create_R_matrix(normalized_test_p1,3)
R_test2 = create_R_matrix(normalized_test_p2,3)
R_test3 = create_R_matrix(normalized_test_p3,3)

## Linear Filter

In [19]:
R_test_1=create_R_matrix(test_features_p1,3) 
pred_finger_lf_p1=np.matmul(R_test_1,f1)

p1_corr_finger1=pearsonr(pred_finger_lf_p1[:,0], Y_test_p1[:,0])
p1_corr_finger2=pearsonr(pred_finger_lf_p1[:,1], Y_test_p1[:,1])
p1_corr_finger3=pearsonr(pred_finger_lf_p1[:,2], Y_test_p1[:,2])
#p1_corr_finger4=pearsonr(pred_finger_lf_p1[:,3], Y_test_p1[:,3])
p1_corr_finger5=pearsonr(pred_finger_lf_p1[:,4], Y_test_p1[:,4])

print("Correlation coefficient for Patient 1 Finger 1:",p1_corr_finger1.statistic)
print("Correlation coefficient for Patient 1 Finger 2:",p1_corr_finger2.statistic)
print("Correlation coefficient for Patient 1 Finger 3:",p1_corr_finger3.statistic)
#print("Correlation coefficient for Patient 1 Finger 4:",p1_corr_finger4.statistic)
print("Correlation coefficient for Patient 1 Finger 5:",p1_corr_finger5.statistic)
avg_corr_coeff = (p1_corr_finger1.statistic + p1_corr_finger2.statistic + p1_corr_finger3.statistic + p1_corr_finger5.statistic)/4
print("Average Correlation coefficient for Patient 1:",avg_corr_coeff)

Correlation coefficient for Patient 1 Finger 1: 0.39522317328756623
Correlation coefficient for Patient 1 Finger 2: 0.5379127158873067
Correlation coefficient for Patient 1 Finger 3: 0.0868640600732274
Correlation coefficient for Patient 1 Finger 5: 0.11308621355032815
Average Correlation coefficient for Patient 1: 0.2832715406996071


In [20]:
R_test_2=create_R_matrix(test_features_p2,3) 
pred_finger_lf_p2=np.matmul(R_test_2,f2)

p2_corr_finger1=pearsonr(pred_finger_lf_p2[:,0], Y_test_p2[:,0])
p2_corr_finger2=pearsonr(pred_finger_lf_p2[:,1], Y_test_p2[:,1])
p2_corr_finger3=pearsonr(pred_finger_lf_p2[:,2], Y_test_p2[:,2])
p2_corr_finger4=pearsonr(pred_finger_lf_p2[:,3], Y_test_p2[:,3])
p2_corr_finger5=pearsonr(pred_finger_lf_p2[:,4], Y_test_p2[:,4])

print("Correlation coefficient for Patient 2 Finger 1:",p2_corr_finger1.statistic)
print("Correlation coefficient for Patient 2 Finger 2:",p2_corr_finger2.statistic)
print("Correlation coefficient for Patient 2 Finger 3:",p2_corr_finger3.statistic)
print("Correlation coefficient for Patient 2 Finger 4:",p2_corr_finger4.statistic)
print("Correlation coefficient for Patient 2 Finger 5:",p2_corr_finger5.statistic)

Correlation coefficient for Patient 2 Finger 1: 0.38147861257849297
Correlation coefficient for Patient 2 Finger 2: 0.22599780076781756
Correlation coefficient for Patient 2 Finger 3: 0.14232231800135578
Correlation coefficient for Patient 2 Finger 4: 0.439253419706396
Correlation coefficient for Patient 2 Finger 5: 0.24842322162033934


## SVR

In [21]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')

svr_pred_p1 = np.zeros((len(R_test1), NUM_FINGERS))
corr_list_p1 = []

for i in range(5):
  svr1=regressor.fit(R_train1,Y_train_p1[:,i])
  svr_pred_p1[:,i] = svr1.predict(R_test1)
  print("Correlation of angles for finger",(i+1),":",  pearsonr(svr_pred_p1[:, i], Y_test_p1[:, i]).statistic)
  corr_list_p1.append(pearsonr(svr_pred_p1[:, i], Y_test_p1[:, i]).statistic)

print("Average coefficient:", (corr_list_p1[0] + corr_list_p1[1] + corr_list_p1[2] + corr_list_p1[4])/4)

Correlation of angles for finger 1 : 0.33923314888668676


KeyboardInterrupt: ignored

In [ ]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')

svr_pred_p2 = np.zeros((len(R_test2), NUM_FINGERS))
corr_list_p2 = []

for i in range(5):
  svr2=regressor.fit(R_train2,Y_train_p2[:,i])
  svr_pred_p2[:,i] = svr2.predict(R_test2)
  print("Correlation of angles for finger",(i+1),":",  pearsonr(svr_pred_p2[:, i], Y_test_p2[:, i]).statistic)
  corr_list_p2.append(pearsonr(svr_pred_p2[:, i], Y_test_p2[:, i]).statistic)

print("Average coefficient:", (corr_list_p2[0] + corr_list_p2[1] + corr_list_p2[2] + corr_list_p2[4])/4)

In [ ]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')

svr_pred_p3 = np.zeros((len(R_test3), NUM_FINGERS))
corr_list_p3 = []

for i in range(5):
  svr3=regressor.fit(R_train3,Y_train_p3[:,i])
  svr_pred_p3[:,i] = svr3.predict(R_test3)
  print("Correlation of angles for finger",(i+1),":",  pearsonr(svr_pred_p3[:, i], Y_test_p3[:, i]).statistic)
  corr_list_p3.append(pearsonr(svr_pred_p3[:, i], Y_test_p3[:, i]).statistic)

print("Average coefficient:", (corr_list_p3[0] + corr_list_p3[1] + corr_list_p3[2] + corr_list_p3[4])/4)

## Fit All Models

In [22]:
def smooth(input, smoothSize):
  #convThing = np.ones(smoothSize)/smoothSize;
  convThing = sig.gaussian(smoothSize, 3)/smoothSize;
  return np.convolve(input, convThing, mode='same')

In [ ]:
p1_f1 = SVR(kernel = 'rbf').fit(R_train1, Y_train_p1[:,0])
p1_f2 = SVR(kernel = 'rbf').fit(R_train1, Y_train_p1[:,1])
p1_f3 = SVR(kernel = 'rbf').fit(R_train1, Y_train_p1[:,2])
p1_f4 = SVR(kernel = 'rbf').fit(R_train1, Y_train_p1[:,3])
p1_f5 = SVR(kernel = 'rbf').fit(R_train1, Y_train_p1[:,4])

p2_f1 = SVR(kernel = 'rbf').fit(R_train2, Y_train_p2[:,0])
p2_f2 = SVR(kernel = 'rbf').fit(R_train2, Y_train_p2[:,1])
p2_f3 = SVR(kernel = 'rbf').fit(R_train2, Y_train_p2[:,2])
p2_f4 = SVR(kernel = 'rbf').fit(R_train2, Y_train_p2[:,3])
p2_f5 = SVR(kernel = 'rbf').fit(R_train2, Y_train_p2[:,4])

p3_f1 = SVR(kernel = 'rbf').fit(R_train3, Y_train_p3[:,0])
p3_f2 = SVR(kernel = 'rbf').fit(R_train3, Y_train_p3[:,1])
p3_f3 = SVR(kernel = 'rbf').fit(R_train3, Y_train_p3[:,2])
p3_f4 = SVR(kernel = 'rbf').fit(R_train3, Y_train_p3[:,3])
p3_f5 = SVR(kernel = 'rbf').fit(R_train3, Y_train_p3[:,4])

In [ ]:
model_list = [[p1_f1, p1_f2, p1_f3, p1_f4, p1_f5],
              [p2_f1, p2_f2, p2_f3, p2_f4, p2_f5],
              [p3_f1, p3_f2, p3_f3, p3_f4, p3_f5]]

In [ ]:
all_corr_list = []
for i in range(len(model_list)):
    for j in range(len(model_list[i])):
        curr_model = model_list[i][j]
        x_test = R_test1
        y_test = Y_test_p1
        if i == 1:
            x_test = R_test2
            y_test = Y_test_p2
        if i == 2:
            x_test = R_test3
            y_test = Y_test_p3
        
        preds = curr_model.predict(x_test)
        preds = smooth(preds,22)
        #print(preds.shape)
        #print(Y_test_p3.shape)
        if j != 3:
            print("Correlation of angles for person", (i+1), "finger",(j+1),":",  pearsonr(preds, y_test[:, j]).statistic)
            all_corr_list.append(pearsonr(preds, y_test[:, j]).statistic)

print("Average Coefficient:", sum(all_corr_list)/len(all_corr_list))

## Retrain models on the entire dataset

### Leaderboard data

In [23]:
# Get all R matrices
# Testing data here needs to be replaced with the leaderboard data

p1_allFeats = get_windowed_feats(total_p1_ecog, fs, window_length, window_overlap)
p1_allFeats = np.array(p1_allFeats)

p2_allFeats = get_windowed_feats(total_p2_ecog, fs, window_length, window_overlap)
p2_allFeats = np.array(p2_allFeats)

p3_allFeats = get_windowed_feats(total_p3_ecog, fs, window_length, window_overlap)
p3_allFeats = np.array(p3_allFeats)

p1_normalized_all, p1_normalized_test = standardize_train_test(p1_allFeats, test_features_p1)
p2_normalized_all, p2_normalized_test = standardize_train_test(p2_allFeats, test_features_p2)
p3_normalized_all, p3_normalized_test = standardize_train_test(p3_allFeats, test_features_p3)

R_p1 = create_R_matrix(p1_normalized_all,3)
R_p2 = create_R_matrix(p2_normalized_all,3)
R_p3 = create_R_matrix(p3_normalized_all,3)

In [24]:
# Get all downsampled Y matrices

# Patient 1

M=NumWins(total_p1_glove[:,0],window_length,window_overlap,fs) #calculate number of windows and remaining time
winLen_samples=round(window_length*fs) #window length in samples
winDisp_samples=round(window_overlap*fs) #window displacement in samples 
window_start_in_windows = round(len(total_p1_glove[:,0])/fs) - (M*window_overlap)
window_start = round(window_start_in_windows*fs)
Y_p1=np.empty([int(M),5])
for mm in range(round(M)): #for each window
  Y_p1[mm,:]=total_p1_glove[window_start,:] #find the flexion data for each finger at that window from glove data for P1
  window_start=window_start+winDisp_samples #move to next window

# Patient 2

M=NumWins(total_p2_glove[:,0],window_length,window_overlap,fs) #calculate number of windows and remaining time
winLen_samples=round(window_length*fs) #window length in samples
winDisp_samples=round(window_overlap*fs) #window displacement in samples 
window_start_in_windows = round(len(total_p2_glove[:,0])/fs) - (M*window_overlap)
window_start = round(window_start_in_windows*fs)
Y_p2=np.empty([int(M),5])
for mm in range(round(M)): #for each window
  Y_p2[mm,:]=total_p2_glove[window_start,:] #find the flexion data for each finger at that window from glove data for P1
  window_start=window_start+winDisp_samples #move to next window

# Patient 3

M=NumWins(total_p3_glove[:,0],window_length,window_overlap,fs) #calculate number of windows and remaining time
winLen_samples=round(window_length*fs) #window length in samples
winDisp_samples=round(window_overlap*fs) #window displacement in samples 
window_start_in_windows = round(len(total_p3_glove[:,0])/fs) - (M*window_overlap)
window_start = round(window_start_in_windows*fs)
Y_p3=np.empty([int(M),5])
for mm in range(round(M)): #for each window
  Y_p3[mm,:]=total_p3_glove[window_start,:] #find the flexion data for each finger at that window from glove data for P1
  window_start=window_start+winDisp_samples #move to next window

In [25]:
fin_p1_f1 = SVR(kernel = 'rbf').fit(R_p1, Y_p1[:,0])
fin_p1_f2 = SVR(kernel = 'rbf').fit(R_p1, Y_p1[:,1])
fin_p1_f3 = SVR(kernel = 'rbf').fit(R_p1, Y_p1[:,2])
fin_p1_f4 = SVR(kernel = 'rbf').fit(R_p1, Y_p1[:,3])
fin_p1_f5 = SVR(kernel = 'rbf').fit(R_p1, Y_p1[:,4])

fin_p2_f1 = SVR(kernel = 'rbf').fit(R_p2, Y_p2[:,0])
fin_p2_f2 = SVR(kernel = 'rbf').fit(R_p2, Y_p2[:,1])
fin_p2_f3 = SVR(kernel = 'rbf').fit(R_p2, Y_p2[:,2])
fin_p2_f4 = SVR(kernel = 'rbf').fit(R_p2, Y_p2[:,3])
fin_p2_f5 = SVR(kernel = 'rbf').fit(R_p2, Y_p2[:,4])

fin_p3_f1 = SVR(kernel = 'rbf').fit(R_p3, Y_p3[:,0])
fin_p3_f2 = SVR(kernel = 'rbf').fit(R_p3, Y_p3[:,1])
fin_p3_f3 = SVR(kernel = 'rbf').fit(R_p3, Y_p3[:,2])
fin_p3_f4 = SVR(kernel = 'rbf').fit(R_p3, Y_p3[:,3])
fin_p3_f5 = SVR(kernel = 'rbf').fit(R_p3, Y_p3[:,4])

In [26]:
fin_model_list = [[fin_p1_f1, fin_p1_f2, fin_p1_f3, fin_p1_f4, fin_p1_f5],
              [fin_p2_f1, fin_p2_f2, fin_p2_f3, fin_p2_f4, fin_p2_f5],
              [fin_p3_f1, fin_p3_f2, fin_p3_f3, fin_p3_f4, fin_p3_f5]]

Testing the final models on the testing dataset

In [ ]:
fin_corr_list = []
for i in range(len(fin_model_list)):
    for j in range(len(fin_model_list[i])):
        curr_model = fin_model_list[i][j]
        x_test = R_test1
        y_test = Y_test_p1
        if i == 1:
            x_test = R_test2
            y_test = Y_test_p2
        if i == 2:
            x_test = R_test3
            y_test = Y_test_p3
        
        preds = curr_model.predict(x_test)
        #preds = smooth(preds,22)
        #print(preds.shape)
        #print(Y_test_p3.shape)
        if j != 3:
            print("Correlation of angles for person", (i+1), "finger",(j+1),":",  pearsonr(preds, y_test[:, j]).statistic)
            fin_corr_list.append(pearsonr(preds, y_test[:, j]).statistic)

print("Average Coefficient:", sum(fin_corr_list)/len(fin_corr_list))

## Leaderboard Data

In [27]:
def zoInterp(x, numInterp):
  repeat=np.tile(x[0],numInterp)
  interpolation_x=repeat
  for p in range(1,len(x)):
    repeat=np.tile(x[p],numInterp)
    interpolation_x=np.hstack((interpolation_x,repeat))
  return interpolation_x

In [28]:
# Get leaderboard data
leaderboard_data = scipy.io.loadmat('leaderboard_data.mat')
p1_leaderboard_ecog=leaderboard_data['leaderboard_ecog'][0][0]
p2_leaderboard_ecog=leaderboard_data['leaderboard_ecog'][1][0]
p3_leaderboard_ecog=leaderboard_data['leaderboard_ecog'][2][0]
p1_ecog=p1_leaderboard_ecog[:,np.arange(62)!=54] #remove channel 55
p2_ecog=p2_leaderboard_ecog[:,np.logical_and(np.arange(48)!=20,np.arange(48)!=37)] #remove channel 21 and 38
p3_ecog=p3_leaderboard_ecog

In [29]:
# Get features from leaderboard data

p1_Feats_l=get_windowed_feats(p1_leaderboard_ecog, fs, window_length, window_overlap)
p2_Feats_l=get_windowed_feats(p2_leaderboard_ecog, fs, window_length, window_overlap)
p3_Feats_l=get_windowed_feats(p3_leaderboard_ecog, fs, window_length, window_overlap)
p1_Feats_l=np.array(p1_Feats_l)
p2_Feats_l=np.array(p2_Feats_l)
p3_Feats_l=np.array(p3_Feats_l)

In [30]:
# Standardize the training and leaderboard data

p1_normalized_all, p1_normalized_leader = standardize_train_test(p1_allFeats, p1_Feats_l)
p2_normalized_all, p2_normalized_leader = standardize_train_test(p2_allFeats, p2_Feats_l)
p3_normalized_all, p3_normalized_leader = standardize_train_test(p3_allFeats, p3_Feats_l)

R_p1_leader=create_R_matrix(p1_normalized_leader,3) #N=3
R_p2_leader=create_R_matrix(p2_normalized_leader,3) #N=3
R_p3_leader=create_R_matrix(p3_normalized_leader,3) #N=3

In [43]:
samples,channels=np.shape(p1_leaderboard_ecog)
leaderboard_pred_3=np.zeros((samples,5))
leaderboard_pred_2=np.zeros((samples,5))
leaderboard_pred_1=np.zeros((samples,5))

In [58]:
rem_samples = round(len(total_p1_ecog[:,0])/fs) - (M*window_overlap)
rem_samples = round(rem_samples*fs)
rem_indices=rem_samples+winLen_samples-winDisp_samples

In [59]:
total_samples = 147500

In [60]:
y_pred_test3 = fin_model_list[2][0].predict(R_p3_leader)
smoothed3=smooth(y_pred_test3,20)
interpAngle3=zoInterp(smoothed3,winDisp_samples)
padding3=np.tile(interpAngle3[0], total_samples - len(interpAngle3) )
totalPrediction3=np.transpose(np.hstack((padding3,interpAngle3)))
#add to prediction array
leaderboard_pred_3[:,0]=totalPrediction3

In [61]:
for i in range(5):
    print("i =", i)
    y_pred_test3 = fin_model_list[2][i].predict(R_p3_leader)
    smoothed3=smooth(y_pred_test3,20)
    interpAngle3=zoInterp(smoothed3,winDisp_samples)
    padding3=np.tile(interpAngle3[0],total_samples - len(interpAngle3))
    totalPrediction3=np.transpose(np.hstack((padding3,interpAngle3)))
    #add to prediction array
    leaderboard_pred_3[:,i]=totalPrediction3

    #Patient 2
    y_pred_test2 = fin_model_list[1][i].predict(R_p2_leader)
    smoothed2=smooth(y_pred_test2,20)

    #interpolate the prediction finger angles for this finger/patient
    interpAngle2=zoInterp(smoothed2,winDisp_samples)
    padding2=np.tile(interpAngle2[0],total_samples - len(interpAngle2))
    totalPrediction2=np.transpose(np.hstack((padding2,interpAngle2)))
    #add to prediction array
    leaderboard_pred_2[:,i]=totalPrediction2

    #Patient 1
    y_pred_test1 = fin_model_list[0][i].predict(R_p1_leader)
    smoothed1=smooth(y_pred_test1,20)

    #interpolate the prediction finger angles for this finger/patient
    interpAngle1=zoInterp(smoothed1,winDisp_samples)
    padding1=np.tile(interpAngle1[0],total_samples - len(interpAngle1))
    totalPrediction1=np.transpose(np.hstack((padding1,interpAngle1)))
    leaderboard_pred_1[:,i]=totalPrediction1

In [62]:
import numpy as np
from scipy.io import savemat

predictions = np.zeros((3,1), dtype=object)
predictions[0,0] = leaderboard_pred_1
predictions[1,0] = leaderboard_pred_2
predictions[2,0] = leaderboard_pred_3

#save the array using the right format
savemat('predictions.mat', {'predicted_dg':predictions})